<a id='start'></a>

# Проект для интернет-магазина (с использованием BERT)

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель исследования:**
Обучить модель классифицировать комментарии на позитивные и негативные (построить модель со значением метрики качества *F1* не меньше 0.75). 

**Задачи исследования:**

1. [Загрузить и подготовить данные;](#step1)
2. [Преобразовать текст из дасасета в векторы с помощью BERT;](#step2)
3. [Обучить различные модели и выбрать наилучшую;](#step3)
4. [Протестировать модель;](#step4)
5. [Сделать вывод.](#step5)


<a id='step1'></a>

## Подготовка

Импорт библиотек

In [3]:
import numpy as np
import pandas as pd

import torch
import transformers as ppb
import warnings
import gc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

from catboost import CatBoostClassifier
from catboost.utils import eval_metric

from transformers import AutoModel, AutoTokenizer

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader

from tqdm import notebook
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

Чтение и проверка данных:

In [5]:
data = pd.read_csv('/datasets/toxic_comments.csv', engine='python', error_bad_lines=False,  encoding='latin-1')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


(в одной из миллиона попыток сделать проект у меня объектов было больше 200к, похоже либо как-то некорректно считывается или загружается датасет)

Пропусков нет, зато есть ненужный столбец, повторяющий индекс, избавимся от него.

In [7]:
data = data.drop(['Unnamed: 0'], axis=1)

Итак, имеем дело с текстами для обучения в столбце 'text' и отметками об оттенке каждого текста, положительный он или токсичный, в столбце с целевым признаком 'toxic'.

Проверим сбалансированность классов в таргете

In [7]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Классы несбалансированы. Пока менять ничего не будем.<br>

In [10]:
data = data.dropna()

In [11]:
corpus = data['text']

<a id='step2'></a>

Инициализируем BERT, который уже обучен работать с целью классификации текстов на токсичные и положительные

Токенайзер отвечает за подготовку входных данных для модели.

In [12]:
model_name = "unitary/toxic-bert" 
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Преобразуем текст в номера токенов из словаря и обрезаем под нужное количество токенов

In [13]:
tokenized = corpus.apply(lambda x: tokenizer.encode(x, max_length=128, truncation=True, add_special_tokens=True))

Добавлением нулей приводим все векторы к одной длине (наибольшей длине вектора, но в нашем случае, как я понял, длине которую задали выше).

In [14]:
padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True)

In [15]:
np.array(padded).shape

(159292, 128)

Создадим маску (укажем, что нули ничего не значат).

In [16]:
attention_mask = padded > 0
attention_mask = attention_mask.type(torch.LongTensor)    

Создаем загрузчик и датасет.

In [17]:
dataset = TensorDataset(attention_mask, padded)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)

Провряем, доступен ли ли GPU

In [18]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") #проверяет доступно ли нам GPU или нет
print(f'Device: {device}')

Device: cuda:0


GPU доступен.

Создаем эмбеддинги и указываем, чтобы вычисления проводились на GPU

In [19]:
embeddings = []
model.to(device)
model.eval() 
for attention_mask, padded in notebook.tqdm(dataloader):
    attention_mask, padded = attention_mask.to(device), padded.to(device)

    with torch.no_grad(): 
        batch_embeddings = model(padded, attention_mask=attention_mask)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) 


  0%|          | 0/4978 [00:00<?, ?it/s]

Разделяем данные на признаки и целевой признак

In [20]:
features = np.concatenate(embeddings)
target = data['toxic']

df_features = pd.DataFrame(features)

features_train, features_test, target_train, target_test = train_test_split(df_features, target)


Почистим память

In [21]:
del data
del attention_mask
del embeddings

gc.collect()
torch.cuda.empty_cache()

<a id='step3'></a>

## Обучение

*Логистическая регрессия*

Сначала обучим логистическую регрессию, перебирать гиперпараметры не будем.

In [ ]:
%%time

model = LogisticRegression()
model.fit(features_train, target_train) 
result = cross_val_score(model, features_train, target_train, cv=5, scoring='f1').mean()
print('F1:', result)

F1: 0.9354544412754944
CPU times: user 2min 8s, sys: 6.66 s, total: 2min 15s
Wall time: 1min 12s


F1 = ~0.94 за 2 мин 15 секунд. На этом можно было бы остановиться.

*Решающее дерево*

Будем варьировать глубину дерева

In [ ]:
%%time

params = { 'max_depth': np.arange(1, 16) }


model = DecisionTreeClassifier(random_state=12345)

tree_grid = GridSearchCV(model, params, scoring='f1')
tree_grid.fit(features_train, target_train)


print('Наилучшие гиперпараметры: '+str(tree_grid.best_params_))
print('Наибольший F1: '+str(abs(tree_grid.best_score_)))

Наилучшие гиперпараметры: {'max_depth': 3}
Наибольший F1: 0.9266595315315808
CPU times: user 1h 45min 35s, sys: 8.02 s, total: 1h 45min 43s
Wall time: 1h 45min 57s


F1 = ~0.93 за 1 час 45 минут, с ЛР не сравнится.

*Случайный лес*

Варьируем количество деревьев и глубину.

In [ ]:
%%time

params = { 'n_estimators' : np.arange(13, 19), 
           'max_depth': np.arange(13, 19) }

model = RandomForestClassifier(random_state=12345)

forest_grid = GridSearchCV(model, params, scoring='f1')
forest_grid.fit(features_train, target_train)


print('Наилучшие гиперпараметры: '+str(forest_grid.best_params_))
print('Наибольший F1: '+str(abs(forest_grid.best_score_)))

Наилучшие гиперпараметры: {'max_depth': 17, 'n_estimators': 17}
Наибольший F1: 0.9331585718192714
CPU times: user 2h 49min, sys: 17 s, total: 2h 49min 17s
Wall time: 2h 49min 39s


F1 = 0.93 за 2 часа 49 минут. Возможно, при увеличении диапазонов получили бы более высокое значение F1 (но времени и нервов потрачено уже достаточно).

*CatBoost*

В КБ варьируем глубину и скорость обучения.

In [24]:
%%time

params = { 'depth' : np.arange(10, 13),
           'learning_rate': np.array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7]) }

model = CatBoostClassifier(random_state=12345, verbose=False, iterations=50, eval_metric='F1')
grid_search_result = model.grid_search(params,
                                             X=features_train,
                                             y=target_train,
                                             cv=5,
                                             partition_random_seed=12345,
                                             calc_cv_statistics=True,
                                             search_by_train_test_split=True,
                                             refit=True,
                                             shuffle=False,
                                             train_size=0.8,
                                             verbose=False )

print('Наилучшие гиперпараметры:', grid_search_result['params'])



bestTest = 0.9370317878
bestIteration = 22


bestTest = 0.9346286177
bestIteration = 25


bestTest = 0.9333333333
bestIteration = 17


bestTest = 0.9336032389
bestIteration = 9


bestTest = 0.9343598055
bestIteration = 14


bestTest = 0.9303452453
bestIteration = 3


bestTest = 0.9355164746
bestIteration = 39


bestTest = 0.9337641357
bestIteration = 5


bestTest = 0.9339010543
bestIteration = 7


bestTest = 0.9313348187
bestIteration = 46


bestTest = 0.9303925536
bestIteration = 2


bestTest = 0.9285569723
bestIteration = 5


bestTest = 0.9362992922
bestIteration = 31


bestTest = 0.9360758217
bestIteration = 7


bestTest = 0.9336569579
bestIteration = 27


bestTest = 0.9324433657
bestIteration = 42


bestTest = 0.9269480519
bestIteration = 1


bestTest = 0.9290322581
bestIteration = 1

Training on fold [0/5]

bestTest = 0.9388347502
bestIteration = 13

Training on fold [1/5]

bestTest = 0.9347064882
bestIteration = 24

Training on fold [2/5]

bestTest = 0.9305870237
bestIteration =

In [26]:
print('Максимальное значение F1:', max(grid_search_result['cv_results']['test-F1-mean']))

Максимальное значение F1: 0.9336422833004241


F1 = 0.93 на катбусте.

Наилучший F1 показала логистическая регрессия. Протестируем модель на тестовой выборке.

<a id='step4'></a>

## Тестирование

In [28]:
%%time

model = LogisticRegression()
model.fit(features_train, target_train)
predict_test = model.predict(features_test)
result = f1_score(target_test, predict_test)

print('F1 на тестовой выборке:', result)

F1 на тестовой выборке: 0.9329460790503852
CPU times: user 25.9 s, sys: 1.13 s, total: 27 s
Wall time: 13.9 s


F1 = 0.93 на тестовой выборке, хороший результат!

<a id='step5'></a>

## Выводы

Таким образом, в ходе работы над проектом:
1. Были загружены и изучены данные, произведена предварительная обработка;
2. Подготовлена модель BERT для определения оттенка текста (токсичный или положительный);
3. Обучены модели линейной регрессии, решающего дерева, случайного леса и CatBoost; подобраны гиперпараметры, для которых F1 на обучающей выборке максимальный;
4. На обучающей выборке наилучший результат показала Логистическая Регрессия;
5. Протестирована модель Логистической Регрессии, показавшая на обучающей выборке лучший результат. На тестовой выборке F1 оказалось равным 0.93, результат можно считать удовлетворительным.

В итоге в качестве модели для классификации текста выбрана Логистическая Регрессия.

<br>

[В начало](#start)